In [4]:
import sys 
sys.path.append('..')
from rul_common.config import ConfigTrain as ct
from rul_common.utils_train import sensors
from rul_common.utils_train import header

import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import OneHotEncoder, LabelEncoder
import warnings

#modeling part
from sklearn.cluster import KMeans
from sklearn.model_selection import RandomizedSearchCV, train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier 
from sklearn.ensemble import RandomForestClassifier
from catboost import CatBoostClassifier, Pool
from lightgbm import LGBMClassifier


from scipy.stats import randint

from sklearn.metrics import accuracy_score, confusion_matrix, precision_score, recall_score, f1_score, classification_report, ConfusionMatrixDisplay
from sklearn.metrics import PrecisionRecallDisplay

import shap

warnings.filterwarnings('ignore')
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', 50)
pd.set_option('display.float_format',  '{:,}'.format)



def read_in(file1, file2, file3, file4):
    df1 = pd.read_csv(file1, sep = " ", header = 0, names = header)
    df1['dataset'] = "FD001"
    df2 = pd.read_csv(file2, sep = " ", header = 0, names = header)
    df2['dataset'] = "FD002"
    df3 = pd.read_csv(file3, sep = " ", header = 0, names = header)
    df3['dataset'] = "FD003"
    df4 = pd.read_csv(file4, sep = " ", header = 0, names = header)
    df4['dataset'] = "FD004"
    return df1, df2, df3, df4

def concatenation_x2(df1, df2):
    train = pd.concat([df1, df2], ignore_index=True)
    train = train.drop(["blank1", "blank2"], axis = 1)
    return train


def remining_cycles(train):
    maxi_df = pd.DataFrame(train.groupby(["esn", "dataset"])["cycles"].max().reset_index())
    maxi_df.rename(columns = {"cycles" : "max_cycles"}, inplace = True)
    train = train.merge(maxi_df, how = "left", on = ["esn", "dataset"])
    train['rul'] = train['cycles'] - train['max_cycles']
    return train

n_clusters = 6
def clustering(train):
    kmeans = KMeans(n_clusters = n_clusters, 
                    random_state = 0, 
                    n_init="auto").fit(train[["opset1","opset2"]].to_numpy())
    train["condition"] = kmeans.labels_
    return train

# lead time is a parameter to 
lead_time = -100
def make_y(row):
    """lead time is a parameter to be decided with what anticipation the business needs to know 
    about a failure to have enough time for action.
    100 cycles is a convenient anticipation

    This function creates Y_target value if ESN is less that 100 cycles to the Failure or more
    """
    return 1 if row["rul"] > lead_time else 0

def one_hot(df, column, names=None):
    enc = OneHotEncoder()
    labelenc = LabelEncoder()
    if names == None:
        names = df[column].unique().tolist()
    df['column_cat'] = labelenc.fit_transform(df[column])
    enc_df = pd.DataFrame(enc.fit_transform(df[column].to_numpy().reshape(-1,1)).toarray(), columns = names)
    enc_df.drop(columns = names[0], inplace = True)
    df = pd.concat([df, enc_df], axis = 1)
    df.drop(columns = column, inplace = True)
    df.drop(columns = 'column_cat', inplace = True)
    return df

In [3]:
dfFD001, dfFD002, dfFD003, dfFD004 = read_in(ct.location1, ct.location2, ct.location3, ct.location4)

In [5]:
trainHPC = concatenation_x2(dfFD001, dfFD003)
trainHPC_Fan = concatenation_x2(dfFD002, dfFD004)
trainHPC = remining_cycles(trainHPC)
trainHPC_Fan = remining_cycles(trainHPC_Fan)
trainHPC = clustering(trainHPC)
trainHPC_Fan = clustering(trainHPC_Fan)

1. Rozdzielnie dwa datasety analizować : a. samo HPC, b. HPC plus Fan
2. model na anomaly detection dla danego Failure Mode (razy dwa bo dwa datasety)
3. wiem, że ostatnie 10 cykli to na pewno chorzy a pierwsze 10 w treningowej to na pewno zdrowi
4. wielka dziura w środku jak zaklasyfikować obserw od 10 do -10
5. z obrazkow widać, że ostatnie N cykli dla każdego silnika to widać zuzycie - chcemy znaleźć N
6. zeby znalexc N spr dla jakiego N dostaniemy najlepszy model klasyfikacyjny.
7. przy jakim N jakość klasyfikacji znacznie spadnie.
8. clustering na anomally detection niemożliwy bo nie są to rozdzielne chmury punktów - ciągle 
9. czemu nie wyznaczyć anomalli na na y na wartość sensora? Bo mamy kilka warunków lotu więc brak jednego uniwersalnego y
10. jeśli warunki na datasety i oper params to rozbimi złożony algo regułowy. 
11. dobieranie tej wartosci recznie to zmudne - trenowanie modeli. Light sprawdza czułość na każdym rozgałezieniu.

przyjmuje lead time z zakresu -150 do -10 zwraca AUC i AP

Jak to zrobię to wybieram najlepszy leadtime = max AP lub AUC

Mamy lead time i model clas czy silnik jest zdrowy czy chory.
i tu gdzie chory to mam prawdopodob, że chory i biorę tę zmienną jako zmienną pomocniczą do liczenia RUL.
Podejscie 1. Dostawiamy kolumnę pred_proba, że chory i liczymy RUL na wszystkim RNN
Podejście 2. filtrujemy po prawdd, że chory, trenujemy RNN tylko na cyyklach chorych i tu, gdzie prawd bardzo małe dopisujemy ręcznie, że RUL większy niż lead time

